# Inference Calculation

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM


In [4]:
import shapely
import pandas as pd
from tqdm import tqdm
import pickle as pkl
from analysis_toolbox import *

In [47]:
DATA_DIR="final_all_tiles"

In [70]:
A_thres=10
SAVE=False
def QjLSA(A_thres,SAVE=False):
    '''
    Large Small Area (LSA) Tile EM Worker model 
    Compute the set of Worker qualities
    A_thres: Area threshold
    '''
    my_BBG  = pd.read_csv("my_ground_truth.csv")
    Qj1=[]
    Qj2=[]
    for object_id in tqdm(list(set(my_BBG.object_id))):
        ground_truth_match = my_BBG[my_BBG.object_id==object_id]
        x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
        T = Polygon(zip(x_locs,y_locs))
        tiles = pkl.load(open(DATA_DIR+"/vtiles{}.pkl".format(object_id)))
        indMat = pkl.load(open(DATA_DIR+"/indMat{}.pkl".format(object_id)))
        workers = pkl.load(open(DATA_DIR+"/worker{}.pkl".format(object_id)))
        for wid,j in zip(workers,range(len(workers))):
            large_Ncorrect=0
            large_Nwrong = 0
            small_Ncorrect=0
            small_Nwrong = 0
            for k in range(len(tiles)): 
                tk = tiles[k]
                ljk = indMat[j][k]

                try:
                    overlap = T.intersection(tk).area/T.area>0.8
                    tjkInT = T.contains(tk) or overlap
                except(shapely.geos.TopologicalError):
                    overlap=True
                    tjkInT = T.contains(tk)
                if (ljk ==1 and tjkInT) or (ljk ==0 and (not tjkInT)):
                    if tk.area>A_thres:
                        large_Ncorrect+=1
                    else: 
                        small_Ncorrect+=1
                elif (ljk ==1 and (not tjkInT)) or (ljk ==0 and tjkInT):
                    if tk.area>A_thres:
                        large_Nwrong+=1
                    else: 
                        small_Nwrong+=1
            q1 = large_Ncorrect/float(large_Ncorrect+large_Nwrong)
            q2 = small_Ncorrect/float(small_Ncorrect+small_Nwrong)

            Qj1.append([object_id,wid,q1])
            Qj2.append([object_id,wid,q2])
    Qj1_tbl = pd.DataFrame(Qj1,columns=["object_id","worker_id","Q1"])
    Qj2_tbl = pd.DataFrame(Qj2,columns=["object_id","worker_id","Q2"])
    Qj = Qj1_tbl.merge(Qj2_tbl)
    if SAVE: 
        pkl.dump(Qj,open("Qj1_Qj2.pkl",'w'))
    return Qj

In [49]:
pkl.dump(Qj1,open("Qj1.pkl",'w'))
pkl.dump(Qj2,open("Qj2.pkl",'w'))

------------------------------------------------------------------------------------------

In [6]:
Qj=pkl.load(open("Qj.pkl",'r'))
Qj1=pkl.load(open("Qj1.pkl",'r'))
Qj2=pkl.load(open("Qj2.pkl",'r'))

In [7]:
def compute_Qj_mean(Qj):
    Qj_mean={}
    for wid, qj in Qj.iteritems():
        Qj_mean[wid]=np.mean(qj)
    return Qj_mean

In [8]:
Qj_mean = compute_Qj_mean(Qj)

In [9]:
for i,obj in enumerate([4,3,12]):
    print i,obj

0 4
1 3
2 12


In [10]:
# pTprimeBasic(Qj_mean,Tprime, BBG)

In [11]:
Qj1_mean = compute_Qj_mean(Qj1)
Qj2_mean = compute_Qj_mean(Qj2)

In [12]:
def pTprimeLSA(objid,Qj1_mean,Qj2_mean,Tprime,T):
    '''
    Area Based Tile EM Worker model 
    Given a tile combination Tprime, compute likelihood of that T'=T
    '''
    workers = pkl.load(open(DATA_DIR+"/worker{}.pkl".format(objid)))
    indicatorMat= pkl.load(open(DATA_DIR+"/indMat{}.pkl".format(objid)))
    for k in Tprime: 
        plk=[]
        for j in range(len(workers)):
            tk = tiles[k]
            ljk = indicatorMat[j][k]
            tjkInT = T.contains(tk) 
            qj1 = Qj1_mean[wid]
            qj2 = Qj1_mean[wid]
            if (ljk ==1 and tjkInT) or (ljk ==0 and (not tjkInT)):
                if tk.area>A_thres:
                    plk.append(qj1)
                else: 
                    plk.append(qj2)
            elif (ljk ==1 and (not tjkInT)) or (ljk ==0 and tjkInT):
                if tk.area>A_thres:
                    plk.append(1-qj1)
                else: 
                    plk.append(1-qj2)
    pTprime=np.product(plk)
    return pTprime

In [13]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from TileEM_plot_toolbox import *
from TileEM_Models import *
gamma_properties=True
DATA_DIR="final_all_tiles"
os.chdir(DATA_DIR)
try: 
    topTilePickHeuristic=sys.argv[1]
except(IndexError):
    topTilePickHeuristic="area"

topk = 40
training_tbl = []
my_BBG  = pd.read_csv("../../my_ground_truth.csv")
import itertools

In [14]:
gamma_properties=False
topTilePickHeuristic="area"

In [41]:
def pTprimeBasic(Qj_mean,Tprime,T):
    '''
    Basic Tile EM Worker model 
    Given a tile combination Tprime, compute likelihood of that T'=T
    '''
    workers = pkl.load(open("worker{}.pkl".format(objid)))
    plk=[]
    for k in Tprime: 
        for j,wid in enumerate(workers):
            tk = tiles[k]
            ljk = indicatorMat[j][k]
            tjkInT = T.contains(tk) #overlap > threshold
            qj = Qj_mean[wid]
            if (ljk ==1 and tjkInT) or (ljk ==0 and (not tjkInT)):
                plk.append(qj)
            elif (ljk ==1 and (not tjkInT)) or (ljk ==0 and tjkInT):
                plk.append(1-qj)
    pTprime=np.product(plk)
    return pTprime

In [19]:
def compute_PR(objid,solnset,tiles):
    '''
    Compute precision recall against ground truth bounding box
    for a given solution set and tile coordinates.
    '''
    try:
        joined_bb,problematic_tiles = join_tiles(solnset,tiles)
    except(ValueError):
        return -1,-1
    ground_truth_match = my_BBG[my_BBG.object_id==objid]
    x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
    BBG = shapely.geometry.Polygon(zip(x_locs,y_locs))
    if problematic_tiles!=[]:
        intersect_area =0
        joined_bb_area =0
        for jbb in joined_bb:
            ia = intersection_area(BBG,jbb)
            intersect_area += ia
            joined_bb_area += jbb.area
    else:
        intersect_area=intersection_area(BBG,joined_bb)
        joined_bb_area =joined_bb.area
    if float(joined_bb_area)!=0:
        precision = intersect_area/float(joined_bb_area)
    else:
        # Empty solution set
        precision = -1
    recall = intersect_area/BBG.area
    #This patches up PR>1 because intersect_area using the slow intersection method can sometimes be larger since overlap area may not be completely deducted, so the estimate is a bit larger than the actual area, but not by much 
    if recall>1:
        recall=1
    if precision>1:
        precision=1
    return precision,recall

In [28]:
DATA_DIR

'TileEM'

In [31]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from TileEM_plot_toolbox import *
from TileEM_Models import *
gamma_properties=False
# DATA_DIR = "output"
topTilePickHeuristic="area"
DATA_DIR="final_all_tiles"
os.chdir(DATA_DIR)
try: 
    topTilePickHeuristic=sys.argv[1]
except(IndexError):
    topTilePickHeuristic="area"

topk = 40
training_tbl = []
my_BBG  = pd.read_csv("../../my_ground_truth.csv")
import itertools

In [39]:
import pickle as pkl

In [42]:
for objid in tqdm(object_lst):
    # print "working on obj",objid
    #Get Tile information for that object
    worker_ids,worker_precision_lst,worker_recall_lst = compute_worker_PR_obj(objid,return_worker_id=True)
    tiles = pkl.load(open("../{0}/vtiles{1}.pkl".format(DATA_DIR,objid),'r'))
    worker_ids = pkl.load(open("../{0}/worker{1}.pkl".format(DATA_DIR,objid),'r'))
    indicatorMat = pkl.load(open("../{0}/indMat{1}.pkl".format(DATA_DIR,objid),'r'))
    if gamma_properties : 
        gammas = pkl.load(open("gfile{}.pkl".format(objid),'r'))
        if  list(gammas[0])==[] and topTilePickHeuristic=='gamma' :
            print "No Gamma information for this object, going onto the next"
            continue
    # Worker Qualities computed by ground truth   
    Qj=pkl.load(open("../Qj.pkl",'r'))
    Qj_mean = compute_Qj_mean(Qj)
    #using the area information in the last row 
    tile_area = np.array(indicatorMat[-1])
    # Loop through all combinations of 20 randomly chosen tiles 
    # tile_subset_idx = np.random.choice(np.arange(len(tiles)),20)
    ground_truth_match = my_BBG[my_BBG.object_id==objid]
    x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
    BBG = shapely.geometry.Polygon(zip(x_locs,y_locs))
    tile_area_ratio = tile_area/BBG.area

    if topTilePickHeuristic=="area": 
        # Loop through all combinations of 40 top-area tiles 
        # print "area heuristic"
        tile_subset_idx = tile_area_ratio.argsort()[::-1][:topk]
    elif topTilePickHeuristic=="gamma":
        # print "gamma"
        tile_subset_idx = np.argsort(gammas[0])[::-1][:topk]
    elif topTilePickHeuristic=="majvote":
        # print "majvote"
        tile_votes = np.sum(indicatorMat[:-1],axis=0)
        tile_subset_idx = np.argsort(tile_votes)[::-1][:topk]

    rand_subset =[]
    for i in range(3000):
        NumTilesInCombo= np.random.randint(0,topk)
        tidxInCombo= np.random.choice(tile_subset_idx,NumTilesInCombo,replace=False)
        rand_subset.append(tidxInCombo)
    # Compute feature properties for T'
    for Tprime in rand_subset:
        p,r =compute_PR(objid,Tprime,tiles)
        gvals=[]
        experiment_idx=0
        # Majority Votes 
        region_votes=[]
        Tareas=[]
        for tidx in Tprime:
            #Number of votes for that tile
            region_votes.append(np.sum(indicatorMat[:-1][:,tidx]))

            if gamma_properties:
                if gammas!=[]:
                    gvals.append(gammas[experiment_idx][tidx])
                else:
                    gvals.append(0)

            Tareas.append(Polygon(tiles[tidx]).area)
        pTprime = pTprimeBasic(Qj_mean,Tprime,BBG)
        training_tbl.append([objid,Tprime,np.sum(region_votes), np.mean(region_votes),np.sum(gvals),np.mean(gvals),np.sum(Tareas),np.mean(Tareas),pTprime,p,r])

# df = pd.DataFrame(training_tbl,columns=["objid","T prime","Total Votes","Average Votes","Total gamma value","Average gamma value",\
#                                         "Total area","Average area","pTprime","Precision","Recall"])
# df.to_csv("tile_combo_metric_{}.csv".format(topTilePickHeuristic))

  0%|          | 0/47 [00:00<?, ?it/s]ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982
ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982
ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982
ERROR:shapely.geos:TopologyException: found non-noded intersection between LINESTRING (384.035 333.596, 380 334) and LINESTRING (382.308 333.769, 377.803 335.521) at 382.30769230769232 333.76923076923077
ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982
ERROR:shapely.geos:TopologyException: found non-noded intersection between LINESTRING (316.737 150.053, 320.667 148.333) and LINESTRING (318.8 149.64, 322.261 147.217) at 320.66666666666669 148.33333333333331
ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982
ERROR:shapely.geos:TopologyException: side locati

KeyboardInterrupt: 

In [44]:
pd.DataFrame(training_tbl,columns=["objid","T prime","Total Votes","Average Votes","Total gamma value","Average gamma value",\
                                         "Total area","Average area","pTprime","Precision","Recall"])

,objid,T prime,Total Votes,Average Votes,Total gamma value,Average gamma value,Total area,Average area,pTprime,Precision,Recall
0,1,"[370, 77, 1755, 3485, 2101, 2428, 1824, 3159, ...",334,22.266667,0,NaN,618.574865,41.238324,2.270536e-151,0.259820,0.031914
1,1,"[3681, 285, 370, 2781, 3336, 371, 1487, 2540, ...",571,20.392857,0,NaN,23738.224924,847.793747,5.143193e-283,0.192805,0.906694
2,1,"[2783, 1755, 3336, 2068, 77, 3159, 3678, 3374,...",509,21.208333,0,NaN,19258.914868,802.454786,3.419580e-245,0.019721,0.075231
3,1,"[3160, 3675, 2783, 2280, 3767, 1755, 77, 2445,...",619,21.344828,0,NaN,19416.408084,669.531313,5.055075e-301,0.021280,0.081665
4,1,"[3162, 3159, 2257, 370, 2280, 3674, 1487, 2101...",766,20.702703,0,NaN,24046.010686,649.892181,0.000000e+00,0.818610,0.879859
5,1,"[2280, 1974, 3675, 2534, 2531, 169, 3161, 2781...",792,21.405405,0,NaN,24006.051098,648.812192,0.000000e+00,0.192702,0.913301
6,1,"[77, 3485, 3336, 2531, 2445, 285, 1974, 3479, ...",755,22.205882,0,NaN,19738.966774,580.557846,0.000000e+00,0.021596,0.084058
7,1,"[3485, 371, 3159, 3767, 3674, 3382, 2531, 2428...",725,22.656250,0,NaN,23938.062915,748.064466,0.000000e+00,0.192506,0.910609
8,1,"[3485, 3479, 2783, 3674, 3382, 2266, 3678, 337...",299,21.357143,0,NaN,4689.993888,334.999563,8.190111e-144,0.900254,0.834740
9,1,"[3681, 3160, 2781, 2266, 3159, 3382, 3336, 337...",285,23.750000,0,NaN,18874.807173,1572.900598,2.198416e-126,0.018234,0.068341
